<a href="https://colab.research.google.com/github/weibb123/RAG_movie_recommender/blob/main/RAG_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [ ]:
!pip install datasets pandas pymongo sentence_transformers
!pip install -U transformers
# Install below if using GPU <- for this notebook, we will be using T4 GPU for cheapest Google Colab compute Units.
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━

## Dataset



In [ ]:
# Load Dataset
from datasets import load_dataset
import pandas as pd

# https://huggingface.co/datasets/AIatMongoDB/embedded_movies
dataset = load_dataset("AIatMongoDB/embedded_movies")

# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset["train"])

dataset_df.head(5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

,cast,num_mflix_comments,countries,directors,title,metacritic,languages,awards,genres,type,writers,plot_embedding,plot,imdb,rated,runtime,poster,fullplot
0,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",0,[USA],"[Louis J. Gasnier, Donald MacKenzie]",The Perils of Pauline,NaN,[English],"{'nominations': 0, 'text': '1 win.', 'wins': 1}",[Action],movie,"[Charles W. Goddard (screenplay), Basil Dickey...","[0.00072939653, -0.026834568, 0.013515796, -0....",Young Pauline is left a lot of money when her ...,"{'id': 4465, 'rating': 7.6, 'votes': 744}",None,199.0,https://m.media-amazon.com/images/M/MV5BMzgxOD...,Young Pauline is left a lot of money when her ...
1,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",0,[USA],"[Alfred J. Goulding, Hal Roach]",From Hand to Mouth,NaN,[English],"{'nominations': 1, 'text': '1 nomination.', 'w...","[Comedy, Short, Action]",movie,[H.M. Walker (titles)],"[-0.022837115, -0.022941574, 0.014937485, -0.0...",A penniless young man tries to save an heiress...,"{'id': 10146, 'rating': 7.0, 'votes': 639}",TV-G,22.0,https://m.media-amazon.com/images/M/MV5BNzE1OW...,As a penniless man worries about how he will m...
2,"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",0,[USA],[Herbert Brenon],Beau Geste,NaN,[English],"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[Action, Adventure, Drama]",movie,"[Herbert Brenon (adaptation), John Russell (ad...","[0.00023330493, -0.028511643, 0.014653289, -0....","Michael ""Beau"" Geste leaves England in disgrac...","{'id': 16634, 'rating': 6.9, 'votes': 222}",None,101.0,None,"Michael ""Beau"" Geste leaves England in disgrac..."
3,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",1,[USA],[Albert Parker],The Black Pirate,NaN,None,"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[Adventure, Action]",movie,"[Douglas Fairbanks (story), Jack Cunningham (a...","[-0.005927917, -0.033394486, 0.0015323418, -0....","Seeking revenge, an athletic young man joins t...","{'id': 16654, 'rating': 7.2, 'votes': 1146}",None,88.0,https://m.media-amazon.com/images/M/MV5BMzU0ND...,A nobleman vows to avenge the death of his fat...
4,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",0,[USA],[Sam Taylor],For Heaven's Sake,NaN,[English],"{'nominations': 1, 'text': '1 nomination.', 'w...","[Action, Comedy, Romance]",movie,"[Ted Wilde (story), John Grey (story), Clyde B...","[-0.0059373598, -0.026604708, -0.0070914757, -...",An irresponsible young millionaire changes his...,"{'id': 16895, 'rating': 7.6, 'votes': 918}",PASSED,58.0,https://m.media-amazon.com/images/M/MV5BMTcxMT...,"The Uptown Boy, J. Harold Manners (Lloyd) is a..."


## Data Preparation

In [ ]:
# Remove data point where plot coloumn is missing
dataset_df = dataset_df.dropna(subset=["fullplot"])
print("\nNumber of missing values in each column after removal:")
print(dataset_df.isnull().sum())

# Remove the plot_embedding from each data point in the dataset as we are going to create new embeddings with an open source embedding model from Hugging Face
dataset_df = dataset_df.drop(columns=["plot_embedding"])
dataset_df.head(5)


Number of missing values in each column after removal:
cast                    1
num_mflix_comments      0
countries               0
directors              12
title                   0
metacritic            893
languages               1
awards                  0
genres                  0
type                    0
writers                13
plot_embedding          1
plot                    0
imdb                    0
rated                 279
runtime                14
poster                 78
fullplot                0
dtype: int64


,cast,num_mflix_comments,countries,directors,title,metacritic,languages,awards,genres,type,writers,plot,imdb,rated,runtime,poster,fullplot
0,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",0,[USA],"[Louis J. Gasnier, Donald MacKenzie]",The Perils of Pauline,NaN,[English],"{'nominations': 0, 'text': '1 win.', 'wins': 1}",[Action],movie,"[Charles W. Goddard (screenplay), Basil Dickey...",Young Pauline is left a lot of money when her ...,"{'id': 4465, 'rating': 7.6, 'votes': 744}",None,199.0,https://m.media-amazon.com/images/M/MV5BMzgxOD...,Young Pauline is left a lot of money when her ...
1,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",0,[USA],"[Alfred J. Goulding, Hal Roach]",From Hand to Mouth,NaN,[English],"{'nominations': 1, 'text': '1 nomination.', 'w...","[Comedy, Short, Action]",movie,[H.M. Walker (titles)],A penniless young man tries to save an heiress...,"{'id': 10146, 'rating': 7.0, 'votes': 639}",TV-G,22.0,https://m.media-amazon.com/images/M/MV5BNzE1OW...,As a penniless man worries about how he will m...
2,"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",0,[USA],[Herbert Brenon],Beau Geste,NaN,[English],"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[Action, Adventure, Drama]",movie,"[Herbert Brenon (adaptation), John Russell (ad...","Michael ""Beau"" Geste leaves England in disgrac...","{'id': 16634, 'rating': 6.9, 'votes': 222}",None,101.0,None,"Michael ""Beau"" Geste leaves England in disgrac..."
3,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",1,[USA],[Albert Parker],The Black Pirate,NaN,None,"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[Adventure, Action]",movie,"[Douglas Fairbanks (story), Jack Cunningham (a...","Seeking revenge, an athletic young man joins t...","{'id': 16654, 'rating': 7.2, 'votes': 1146}",None,88.0,https://m.media-amazon.com/images/M/MV5BMzU0ND...,A nobleman vows to avenge the death of his fat...
4,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",0,[USA],[Sam Taylor],For Heaven's Sake,NaN,[English],"{'nominations': 1, 'text': '1 nomination.', 'w...","[Action, Comedy, Romance]",movie,"[Ted Wilde (story), John Grey (story), Clyde B...",An irresponsible young millionaire changes his...,"{'id': 16895, 'rating': 7.6, 'votes': 918}",PASSED,58.0,https://m.media-amazon.com/images/M/MV5BMTcxMT...,"The Uptown Boy, J. Harold Manners (Lloyd) is a..."


## Generate Embeddings

Get embedding on the full_plots columns(transforming text into numerical word vectors) usin opensource embedding.

In [ ]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")


def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()


dataset_df["embedding"] = dataset_df["fullplot"].apply(get_embedding)

dataset_df.head()

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

,cast,num_mflix_comments,countries,directors,title,metacritic,languages,awards,genres,type,writers,plot,imdb,rated,runtime,poster,fullplot,embedding
0,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",0,[USA],"[Louis J. Gasnier, Donald MacKenzie]",The Perils of Pauline,NaN,[English],"{'nominations': 0, 'text': '1 win.', 'wins': 1}",[Action],movie,"[Charles W. Goddard (screenplay), Basil Dickey...",Young Pauline is left a lot of money when her ...,"{'id': 4465, 'rating': 7.6, 'votes': 744}",None,199.0,https://m.media-amazon.com/images/M/MV5BMzgxOD...,Young Pauline is left a lot of money when her ...,"[-0.009285839274525642, -0.005062091629952192,..."
1,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",0,[USA],"[Alfred J. Goulding, Hal Roach]",From Hand to Mouth,NaN,[English],"{'nominations': 1, 'text': '1 nomination.', 'w...","[Comedy, Short, Action]",movie,[H.M. Walker (titles)],A penniless young man tries to save an heiress...,"{'id': 10146, 'rating': 7.0, 'votes': 639}",TV-G,22.0,https://m.media-amazon.com/images/M/MV5BNzE1OW...,As a penniless man worries about how he will m...,"[-0.002439370146021247, 0.023095937445759773, ..."
2,"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",0,[USA],[Herbert Brenon],Beau Geste,NaN,[English],"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[Action, Adventure, Drama]",movie,"[Herbert Brenon (adaptation), John Russell (ad...","Michael ""Beau"" Geste leaves England in disgrac...","{'id': 16634, 'rating': 6.9, 'votes': 222}",None,101.0,None,"Michael ""Beau"" Geste leaves England in disgrac...","[0.012204294092953205, -0.011455751955509186, ..."
3,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",1,[USA],[Albert Parker],The Black Pirate,NaN,None,"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[Adventure, Action]",movie,"[Douglas Fairbanks (story), Jack Cunningham (a...","Seeking revenge, an athletic young man joins t...","{'id': 16654, 'rating': 7.2, 'votes': 1146}",None,88.0,https://m.media-amazon.com/images/M/MV5BMzU0ND...,A nobleman vows to avenge the death of his fat...,"[0.004541350528597832, -0.0006100559257902205,..."
4,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",0,[USA],[Sam Taylor],For Heaven's Sake,NaN,[English],"{'nominations': 1, 'text': '1 nomination.', 'w...","[Action, Comedy, Romance]",movie,"[Ted Wilde (story), John Grey (story), Clyde B...",An irresponsible young millionaire changes his...,"{'id': 16895, 'rating': 7.6, 'votes': 918}",PASSED,58.0,https://m.media-amazon.com/images/M/MV5BMTcxMT...,"The Uptown Boy, J. Harold Manners (Lloyd) is a...","[-0.002225600415840745, 0.011567802168428898, ..."


Let's see what genres are available for us to see.

We would create an user query to ask for recommendation at the end.

## DATABASE Setup

We will use MONGODB as a vector database to build our RAG system.

steps:
1. Create a project -> Create a database cluster -> connect using drivers -> obtained the URI for your cluster
2. Add Data -> Create DB on Atlas -> Create Vector Search Index

Creating a Vector Search Index enables the ability to traverse the documents efficiently to retrieve documents with embeddings that match the query embedding based on vector similarity.
```
{
 "fields": [{
     "numDimensions": 1024,
     "path": "embedding",
     "similarity": "cosine",
     "type": "vector"
   }]
}```

Insert this JSON-like document to Collection to the database to create Vector Search Index.

The 1024 value of the numDimension field corresponds to the dimension of the vector generated by the gte-large embedding model. If you use the gte-base or gte-small embedding models, the numDimension value in the vector search index must be set to 768 and 384, respectively.

## Create Data Connection

Next, we would like to create connection to the cluster we create and enabling access to its databases and collections.

In [24]:
import pymongo
from pymongo.server_api import ServerApi


def mongo_client(mongo_url):
  """establish connection to the MONGODB"""
  try:
    client = pymongo.MongoClient(mongo_url, server_api=ServerApi('1'))
    client.admin.command('ping')
    print("successful")
    return client
  except Exception as e:
    print(f"connection failed{e}")
    return None

mongo_uri = 'YOUR_mongo_uri'

client = mongo_client(mongo_uri)

# Ingest data into MongoDB
db = client["movies"] # database name we created
collection = db["movie_collection"]# collection name we created

successful


Adding data to mongodb from python dataframe is an easy process. All we need to do is to convert dataframe into dictionary then use **insert_many** method on the collection to pass the converted dataset records.

In [25]:
documents = dataset_df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


you can verify if data has been added by going to MONGODB -> Cluster0 -> Collections

In [29]:
print(db)

Database(MongoClient(host=['ac-triksyk-shard-00-02.rottzct.mongodb.net:27017', 'ac-triksyk-shard-00-01.rottzct.mongodb.net:27017', 'ac-triksyk-shard-00-00.rottzct.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-12lml1-shard-0', tls=True, server_api=<pymongo.server_api.ServerApi object at 0x7d118513cd00>), 'movies')


## Vector Search on User Queries

Next, implement a function that returns a vector search result by generating a query embedding and defining a MongoDB aggregation pipeline.

The pipeline, consisting of the ```vectorSearch``` and ```project``` stages, executes queries using the generated vector and formats the results to include only the required information, such as plot, title, and genres while incorporating a search score for each result.

In [49]:
def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 50,  # Number of candidate matches to consider
                "limit": 5,  # Return top 5 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "fullplot": 1,  # Include the plot field
                "title": 1,  # Include the title field
                "genres": 1,  # Include the genres field
                "score": {"$meta": "vectorSearchScore"},  # Include the search score
            }
        },
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

In [50]:
def get_search_result(query, collection):
  """
  Function to append results in a large text string.
  """

  get_knowledge = vector_search(query, collection)

  search_result = "" # return a string
  for result in get_knowledge:
      search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('fullplot', 'N/A')}\n"

  return search_result

In [51]:
# Conduct query with retrival of sources
query = "What is the best action movie to watch and why?"
source_information = get_search_result(query, collection)
combined_information = f"Query: {query}\nContinue to answer the query by using the Search Results:{source_information}"

print(source_information)

In [37]:
# log in to huggingface to use notebook_login
# need to accept terms of agreement to use Google's Gemma
from huggingface_hub import notebook_login

notebook_login()

## Handle User queries load Gemma

Google's new open source model Gemma

In [38]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")

# GPU Enabled use below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [52]:
# Moving tensors to GPU
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=500)
print(tokenizer.decode(response[0]))

<bos>Query: What is the best action movie to watch and why?
Continue to answer the query by using the Search Results:

**Search Results:**

* **The Raid (2011):** A high-octane action film about a group of criminals taking on a corrupt police department.
* **John Wick (2014):** A revenge-driven action film about a retired hitman seeking revenge for his murdered family.
* **Mission: Impossible - Rogue Nation (2018):** A high-octane action film about a group of spies on the run from the CIA.
* **Deadpool (2016):** A superhero action film with a unique and witty take on the genre.
* **Fast and Furious (2006):** A high-octane action film about a group of car enthusiasts taking on a rival street gang.

**Answer:**

The best action movie to watch is **The Raid (2011)**. It's a high-octane action film about a group of criminals taking on a corrupt police department. The film is well-directed, has great action sequences, and features a strong cast of characters. If you're looking for a movie t

Success!!!